In [6]:
import os
import google.generativeai as genai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
%pip install --upgrade --quiet  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set the API key from environment variable
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [8]:
loader = PyPDFLoader("/content/Healthcare_Guide_Proper.pdf")
pages = loader.load()

In [4]:
%pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [5]:
%pip install --upgrade --quiet pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 8.8 MB/s eta 0:00:00


In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(pages)

# 4. Create embeddings using Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [12]:
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

# 6. Create QA chain using Gemini Pro
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatGoogleGenerativeAI(model="gemini-2.5-pro"),
    retriever=retriever,
    return_source_documents=True
)

In [11]:
%pip install --upgrade --quiet faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.9 MB/s eta 0:00:00


In [ ]:

while True:
    query = input("\nAsk your question (or type 'exit'): ")
    if query.lower() == "exit":
        break

    result = qa_chain(query)
    print("\n🔍 Answer:\n", result["result"])


Ask your question (or type 'exit'): Give me Nutrition Tips?


/tmp/ipython-input-13-2999498574.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)



🔍 Answer:
 Based on the provided guide, here are the nutrition tips:

*   Eat a balanced diet rich in fruits, vegetables, whole grains, and lean proteins.
*   Limit intake of processed foods, sugar, and salt.
*   Stay hydrated with plenty of water.

Ask your question (or type 'exit'): how should I maintaince cleanliness?

🔍 Answer:
 Based on the information provided, you should maintain cleanliness by washing your hands frequently and maintaining personal hygiene.

Ask your question (or type 'exit'): Who is elon musk?

🔍 Answer:
 I don't know. The provided text is a healthcare guide and does not contain information about Elon Musk.
